In [1]:
import pandas as pd
from copy import deepcopy

In [3]:
df_copy = pd.read_csv("./cc12m.tsv", sep="\t")

In [7]:
df = deepcopy(df_copy)

In [11]:
df.columns = ["url", "text"]
df.dropna(inplace=True, subset=["text"])
df.drop_duplicates(inplace=True, subset=["text"])

In [15]:
df.to_parquet("./cc12m.parquet", index=False)

In [1]:
import os
import glob
import torch
from PIL import Image
from tqdm import tqdm

In [2]:
img_paths = glob.glob("./images/*.png")

In [13]:
images = []
for img_path in tqdm(img_paths):
    try:
        img = Image.open(img_path).convert("RGB")
        images.append(img)
    except:
        pass

100%|██████████| 12115/12115 [01:35<00:00, 127.22it/s]


In [14]:
import torchvision.transforms as T

size = 55
transform = T.Compose([
    T.Resize(size),
    T.CenterCrop(size),
    T.ToTensor(),
    T.Lambda(lambda x: x * 2 - 1)
])

tensors = []
for img in tqdm(images):
    try:
        tensor = transform(img)
        tensors.append(tensor)
    except:
        pass

  0%|          | 0/11740 [00:00<?, ?it/s]

100%|██████████| 11740/11740 [00:34<00:00, 340.56it/s]


In [16]:
tensors = torch.stack(tensors)

In [17]:
import matplotlib.pyplot as plt

In [60]:
fft = torch.fft.fft2(tensors)
fft = fft.abs()
fft = fft + 1e-6
fft = torch.log(fft)
fft_mean = fft.mean(dim=0)

In [63]:
torch.save(fft_mean, "./fft_mean.pt")

In [62]:
fft_mean.shape

torch.Size([3, 55, 55])

In [66]:
fft_mean.isnan().sum()

tensor(0)